In [48]:
import re
import qiskit
import json
import time

import numpy as np
from typing import List
from qiskit import transpile
import time

import numpy as np

import os
from copy import deepcopy
current_dir=os.getcwd()
# os.chdir("../vqe_tetris/vqe_tetris-master/core")
# from utils import synthesis_lookahead, hardware, synthesis_max_cancel
# os.chdir(current_dir)
os.chdir("../Paulihedral_v2/Paulihedral_new")
from benchmark.mypauli import pauliString
import synthesis_FT, parallel_bl
# from tools import print_qc
os.chdir(current_dir)

In [49]:
def paulihedral(parr):
    nq = len(parr[0][0])
    length = nq//2 # `length' is a hyperparameter, and can be adjusted for best performance
    # a1 = parallel_bl.depth_oriented_scheduling(parr, length=length, maxiter=30)
    a1 = parallel_bl.gate_count_oriented_scheduling(parr)

    qc = synthesis_FT.block_opt_FT(a1)

    return qc

def is_all_identity(pauli):
    return all(char == 'I' for char in pauli)

# def update_json_file(filename, runtime, cx_count, circ_depth):
#     with open(f'benchmarks/results/test_' + filename, 'r') as paulis_file:
#         data=json.load(paulis_file)
#     print(data[1])
#     # with open(f'benchmarks/results/test_' + filename, 'w') as paulis_file:
#     #     json.dump([test_paulis, results], paulis_file, indent=4)

In [50]:
#Compare json files in a specific folder
import os, json
def run_experiment_folder(folder_path = None, filename = None, save_output = False):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)
            # Filter the list to remove all identity Paulis
            test_paulis = [[pauliString(pauli)] for pauli in paulis if not is_all_identity(pauli)]
            number_of_ham=len(test_paulis)
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for method
            start_time = time.time()
            circuit = paulihedral(test_paulis)
            circuit = transpile(circuit, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
            end_time = time.time()
            tot_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": number_of_ham,
                "times": {
                    # "our_time": our_time,
                    # "combined_time": combined_time,
                    # "qiskit_time": qiskit_time,
                    # "rustiq_time": rustiq_time
                    "paulihedral_time": tot_time
                },
                "gate_counts": {
                    # "our_method": opt_qc_f.count_ops().get('cx', 0),
                    # "combined_method": opt_qiskit.count_ops().get('cx', 0),
                    # "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                    # "rustiq_method": entangling_count(circuit),
                    "paulihedral_method": circuit.count_ops().get('cx', 0)
                },
                "circuit_depth": {
                    # "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    # "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    # "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    # "rustiq_method": entangling_depth(circuit),
                    "paulihedral_method": circuit.depth()
                },
                "test_paulis_file": f'benchmarks/results/test_paulihedral_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results/test_paulihedral_' + filename, 'w') as paulis_file:
                    json.dump([paulis, results], paulis_file, indent=4)
    

    

In [51]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, filename = "Paulis"):

    results = []
    # paulis = test_paulis
    # Filter the list to remove all identity Paulis
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]
    
    number_of_ham=len([p for block in test_paulis for p in block])
    # Measure time for method
    paulis=deepcopy(test_paulis)
    paulis = [[pauliString(p) for p in block] for block in paulis]
    start_time = time.time()
    print(paulis)
    circuit= paulihedral(paulis)
    # print(circuit)
    circuit = transpile(circuit, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
    # print(circuit)
    end_time = time.time()
    tot_time = end_time - start_time

    # Collect results
    result = {
        "num_paulis": number_of_ham,
        "times": {
            # "our_time": our_time,
            # "combined_time": combined_time,
            # "qiskit_time": qiskit_time,
            # "rustiq_time": rustiq_time
            "paulihedral_time": tot_time
        },
        "gate_counts": {
            # "our_method": opt_qc_f.count_ops().get('cx', 0),
            # "combined_method": opt_qiskit.count_ops().get('cx', 0),
            # "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            # "rustiq_method": entangling_count(circuit),
            "paulihedral_method": circuit.count_ops().get('cx', 0)
        },
        "circuit_depth": {
            # "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            # "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            # "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            # "rustiq_method": entangling_depth(circuit),
            "paulihedral_method": circuit.depth()
        },
        "test_paulis_file": f'benchmarks/results/test_paulihedral_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results/test_paulihedral_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return circuit #sorted_entanglers_f
    

    

In [52]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,]# 12]
orbitals_list = [4, 6, 8, 12, 16, 20]#, 24]
# paulis_len=[24, 80, 320, 1656, 5376, 13400]
#First evaluate the UCCSD ansatz:
for e, o in zip(electrons_list, orbitals_list):
    filename=f"uccsd_hamiltonian_e{e}_o{o}.json"
    with open(f"benchmarks/uccsd_hamiltonians/{filename}", "r") as file:
        data=json.load(file)
    # test_paulis = generate_UCCSD_entanglers(x, y)
    print(data)
    num_qubits=len(data[0][0])
    data = [[p for p in block if not is_all_identity(p)] for block in data]
    data=[block for block in data if len(block)>0]
    num_hams=len([p for block in data for p in block])
    print(data)
    entanglers = run_experiment_paulis(data, num_qubits, save_output = True, filename=f"Paulis{num_hams}.json")

[['YZXI', 'XZXI', 'YZYI', 'XZYI'], ['IYZX', 'IXZX', 'IYZY', 'IXZY'], ['YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX', 'YYXX', 'XYXX', 'YXXY', 'XXXY', 'YYXY', 'XYXY']]
[['YZXI', 'XZXI', 'YZYI', 'XZYI'], ['IYZX', 'IXZX', 'IYZY', 'IXZY'], ['YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX', 'YYXX', 'XYXX', 'YXXY', 'XXXY', 'YYXY', 'XYXY']]
[[YZXI, XZXI, YZYI, XZYI], [IYZX, IXZX, IYZY, IXZY], [YXYX, XXYX, YYYX, XYYX, YXYY, XXYY, YYYY, XYYY, YXXX, XXXX, YYXX, XYXX, YXXY, XXXY, YYXY, XYXY]]
{'num_paulis': 24, 'times': {'paulihedral_time': 0.38996338844299316}, 'gate_counts': {'paulihedral_method': 13}, 'circuit_depth': {'paulihedral_method': 36}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_Paulis24.json'}
[['YZXIII', 'XZXIII', 'YZYIII', 'XZYIII'], ['YZZZXI', 'XZZZXI', 'YZZZYI', 'XZZZYI'], ['IYZXII', 'IXZXII', 'IYZYII', 'IXZYII'], ['IYZZZX', 'IXZZZX', 'IYZZZY', 'IXZZZY'], ['YXYXII', 'XXYXII', 'YYYXII', 'XYYXII', 'YXYYII',

In [ ]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True)

benzene.json
{'num_paulis': 1254, 'times': {'paulihedral_time': 212.84998846054077}, 'gate_counts': {'paulihedral_method': 4280}, 'circuit_depth': {'paulihedral_method': 8441}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_benzene.json'}
H2O.json
{'num_paulis': 184, 'times': {'paulihedral_time': 18.23982572555542}, 'gate_counts': {'paulihedral_method': 493}, 'circuit_depth': {'paulihedral_method': 1089}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_H2O.json'}
LiH.json
{'num_paulis': 61, 'times': {'paulihedral_time': 4.47067666053772}, 'gate_counts': {'paulihedral_method': 133}, 'circuit_depth': {'paulihedral_method': 335}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_LiH.json'}


In [ ]:
# #Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True)

max_cut_benchmark_n10_e45_l20.json
{'num_paulis': 1100, 'times': {'paulihedral_time': 10.529120206832886}, 'gate_counts': {'paulihedral_method': 90}, 'circuit_depth': {'paulihedral_method': 115}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_max_cut_benchmark_n10_e45_l20.json'}
max_cut_benchmark_n14_e60_l5.json


TranspilerError: 'Maximum iteration reached. max_iteration=1000'

In [ ]:
# #Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = True)

0.24.1
